In [1]:
import numpy as np 
import pandas as pd 
import mlflow
import mlflow.catboost
from catboost import CatBoostRegressor
from mlflow.models import infer_signature
from mlflow.tracking import MlflowClient
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error

import optuna
from optuna.integration.mlflow import MLflowCallback
import joblib

from typing import Optional,Dict,Tuple
from pathlib import Path
from enefit_challenge.utils.dataset import load_enefit_training_data

from enefit_challenge.models.forecaster import Forecaster
import warnings
warnings.filterwarnings('ignore')

/Users/dylantartarini/miniconda3/envs/enefit/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
class CatBoostForecaster(Forecaster):
    def __init__(self)-> None:
        self.tracking_uri = mlflow.set_tracking_uri("http://127.0.0.1:5000/")
        pass

    def fit_model(
        self,  
        X:pd.DataFrame,
        y:pd.Series,
        categorical_features: list=[],
        params:Optional[Dict]=None,
    ) -> CatBoostRegressor:
        """
        Trains a `CatBoostRegressor` with a L1 loss and tracks it using mlflow
        """
        model = CatBoostRegressor(
            n_estimators=100, 
            objective='MAE',
            thread_count=1,
            bootstrap_type =  "Bernoulli",
            sampling_frequency= 'PerTree',
            verbose=0,
            cat_features=categorical_features,
            leaf_estimation_iterations=1
        )
        if params:
            model.set_params(**params)

        model.fit(X, y)
    
        return model
    
    def fit_and_test_fold(
        self, 
        params:Dict,
        X: pd.DataFrame, 
        y: pd.Series, 
        year_month_train, 
        year_month_test,
        categorical_features: list=[],
        experiment_name: str="catboost",
        artifact_path: str="catboost_model",
        metrics: list=["mae"]
    ) -> float:
        
        first_dates_month = pd.to_datetime(X[['year', 'month']].assign(day=1))
        train_index = first_dates_month.isin(year_month_train)
        test_index = first_dates_month.isin(year_month_test)
        X_train = X[train_index];X_test = X[test_index]
        y_train = y[train_index]; y_test = y[test_index]
        # fit model on training data
        model = self.fit_model(
            X_train, 
            y_train, 
            categorical_features,
            params
        )
        # generate predictions
        y_test_pred = model.predict(X_test)
        self.signature = infer_signature(X_train, y_test_pred)
        mae = mean_absolute_error(y_test, y_test_pred)

        mlflow.catboost.log_model(
            model, 
            artifact_path=artifact_path,
            signature=self.signature
        )
        mlflow.log_params(params)

        return mae

    def train_model(
        self, 
        train_df: pd.DataFrame, 
        target_col: str,
        model_name: str,
        exclude_cols: list=[],
        categorical_features: list=[],
        experiment_name: str="catboost",
        artifact_path: str="catboost_model",
        params: Optional[Dict]=None,
        metrics: list=["MAE"]
    ) -> Dict:
        """ 
        Takes an instance of `CatBoostRegressor` model and tracks the hyperparameter tuning
        experiment on training set using `mlflow` and `optuna`.  
        Registers the best version of the model according to a specified metric
        
        -------     
        params:
        -------
        `experiment_name`: `str`
            the name of the experiment used to store runs in mlflow, 
            as well as the name of the optuna study
        `model_name`: `str`
            the name the final model will have in the registry
        `train_df`: `pd.DataFrame`
            the training data for the model.
        `target_col`: `str`
            the time-series target column
        `exclude_cols`: `list`  
            columns in dataset that should not be used
        `categorical_features`: `list`
            list of categorical features in the dataset
        `artifact_path`: `str`
            the path pointing to the mlflow artifact
        `metrics`: `list`
            list of the metrics to track in the mlflow experiment run.
        `params`: `Optional[Dict]`
            optional dictionary of parameters to use
        """
        self.model_name = model_name
        X = train_df.drop([target_col] + exclude_cols, axis=1)
        y = train_df[target_col]
        # unique year-month combinations -> to be used in cross-validation
        timesteps = np.sort(np.array(
            pd.to_datetime(X[['year', 'month']].assign(day=1)).unique().tolist()
        ))

        # define mlflow callback Handler for optuna 
        mlflc = MLflowCallback(
            tracking_uri=self.tracking_uri,
            metric_name="MAE",
        )
    
        @mlflc.track_in_mlflow() # decorator to allow mlflow logging
        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 50, 200),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.95,log=True),
                'depth': trial.suggest_int('depth', 3, 10, log=True),
                'l2_leaf_reg': trial.suggest_float('l2_leaf_reg',1e-8,100,log=True),
                'model_size_reg': trial.suggest_float('model_size_reg',1e-8,100,log=True),
                'colsample_bylevel': trial.suggest_float("colsample_bylevel", 0.1, 1),
                'subsample': trial.suggest_float("subsample", 0.5, 1)
            }
            cv = TimeSeriesSplit(n_splits=3) # cross validation
            cv_mae = [None]*3
            for i, (train_index, test_index) in enumerate(cv.split(timesteps)):
                cv_mae[i] = self.fit_and_test_fold(
                    params,
                    X, 
                    y, 
                    timesteps[train_index], 
                    timesteps[test_index],
                    categorical_features
                )
            trial.set_user_attr('split_mae', cv_mae)
            return np.mean(cv_mae)

        
        sampler = optuna.samplers.TPESampler(
            n_startup_trials=10, 
            seed=42
        )

        self.study = optuna.create_study(
            directions=['minimize'],
            sampler=sampler,
            study_name=experiment_name
        )

        self.study.optimize(objective, n_trials=2, timeout= 3600, callbacks=[mlflc]) 
        
        # # search for the best run at the end of the experiment
        # best_run = mlflow.search_runs(max_results=1,order_by=["metrics.MAE"]).run_id
        # # register new model version in mlflow
        # self.result = mlflow.register_model(
        #     model_uri=f"runs:/{best_run}/{artifact_path}",
        #     name=self.model_name
        # )

    def forecast(
        self, 
        input_data: pd.DataFrame
        ) -> pd.DataFrame:
        """ 
        Fetches the latest version of the model from the mlflow backend and uses it
        to perform prediction on new input data.
        """
        model = mlflow.pyfunc.load_model(
            model_uri=f"models:/{self.model_name}/{self.result.version}"
        )
        y_pred = model.predict(input_data)

        return y_pred
        

In [3]:
df_train = load_enefit_training_data()

In [6]:
not_feature_columns = ['datetime', 'row_id','prediction_unit_id','date','time', 'data_block_id']
cat_columns = ['county', 'product_type']

cbf = CatBoostForecaster()
cbf.train_model(
    train_df=df_train,
    target_col="target",
    model_name="catboost_enefit",
    exclude_cols=not_feature_columns,
    categorical_features=cat_columns,
)

[I 2023-11-12 23:56:43,286] A new study created in memory with name: catboost
[I 2023-11-12 23:57:57,501] Trial 0 finished with value: 126.9517456757764 and parameters: {'n_estimators': 106, 'learning_rate': 0.7590145927293601, 'depth': 7, 'l2_leaf_reg': 0.009695826644515229, 'model_size_reg': 3.6323392569431376e-07, 'colsample_bylevel': 0.2403950683025824, 'subsample': 0.5290418060840998}. Best is trial 0 with value: 126.9517456757764.
[I 2023-11-13 00:00:41,074] Trial 1 finished with value: 84.07288468262263 and parameters: {'n_estimators': 180, 'learning_rate': 0.15446829333828344, 'depth': 7, 'l2_leaf_reg': 1.6063676259174453e-08, 'model_size_reg': 50.014798288569374, 'colsample_bylevel': 0.8491983767203796, 'subsample': 0.6061695553391381}. Best is trial 1 with value: 84.07288468262263.
Registered model 'catboost_enefit' already exists. Creating a new version of this model...


RestException: INVALID_PARAMETER_VALUE: Invalid run ID: '0    95738294fb4245adb5bb6d765b72238cName: run_id, dtype: object'

### Testing

In [ ]:
import mlflow
logged_model = 'runs:/3d2d010927a84b01bf360d76c616ba61/catboost_model'

loaded_model = mlflow.pyfunc.load_model(logged_model)

In [ ]:
not_feature_columns = ['datetime', 'row_id','prediction_unit_id','date','time', 'data_block_id']

X = df_train.drop(['target'] + not_feature_columns, axis=1)
y = df_train['target']

In [ ]:
# Sanity check
y_pred = loaded_model.predict(X)
train_mae = mean_absolute_error(y, y_pred)
print(f'Train MAE: {train_mae:.3f}')